In [11]:
import sympy as sp
import numpy as np
from sympy import Symbol, hessian, simplify, count_ops
from functools import reduce
from sympy.utilities import codegen
from sympy.codegen.rewriting import optimize, optims_c99, create_expand_pow_optimization
import os

def jacobian_for(expr, vars):
    return sp.Matrix([expr]).jacobian(vars)

def hessian_for(expr, vars):
    return hessian(expr, vars)

def generate_code_for(expr, vars, name):
    oe = optimize(expr, optims_c99)
    jac = jacobian_for(oe, vars)
    hes = hessian_for(oe, vars)
    print("Value ops: ", count_ops(oe, True))
    print("Jacob ops: ", count_ops(jac, True))
    # print("Hessi ops: ", count_ops(hes, True))
    cc = codegen.C99CodeGen(project = "acgipc", cse=True)
    [(cn, cc), (hn, hc)] = codegen.codegen(
        [(name + "_value", oe),
         (name + "_grad", jac),
        #  (name + "_hessian", hes)
         ], code_gen=cc)
    if not os.path.exists('generated'):
        os.mkdir('generated')
    with open("generated/" + cn, mode='w') as f:
        f.write(cc)
    with open("generated/" + hn, mode='w') as f:
        f.write(hc)



# Vertex Face Distance

In [12]:
def declare_vector(name, dim):
    return sp.Matrix(sp.symbols([f'{name}_{i}' for i in range(dim)], real=True))

a = declare_vector('a', 3)
b = declare_vector('b', 3)
c = declare_vector('c', 3)
d = declare_vector('d', 3)
free_symbols = reduce(lambda x, y: x + y, [a.tolist(), b.tolist(), c.tolist(), d.tolist()])
free_symbols = reduce(lambda x, y: x + y, free_symbols)
free_symbols

[a_0, a_1, a_2, b_0, b_1, b_2, c_0, c_1, c_2, d_0, d_1, d_2]

In [13]:
normal = (b - a).cross(c - a)
normal_normal = normal.normalized()
off = (d - a)
distance = simplify(normal_normal.dot(off))
distance

(-(a_0 - d_0)*((a_1 - b_1)*(a_2 - c_2) - (a_1 - c_1)*(a_2 - b_2)) + (a_1 - d_1)*((a_0 - b_0)*(a_2 - c_2) - (a_0 - c_0)*(a_2 - b_2)) - (a_2 - d_2)*((a_0 - b_0)*(a_1 - c_1) - (a_0 - c_0)*(a_1 - b_1)))/sqrt(((a_0 - b_0)*(a_1 - c_1) - (a_0 - c_0)*(a_1 - b_1))**2 + ((a_0 - b_0)*(a_2 - c_2) - (a_0 - c_0)*(a_2 - b_2))**2 + ((a_1 - b_1)*(a_2 - c_2) - (a_1 - c_1)*(a_2 - b_2))**2)

In [14]:
generate_code_for(distance, free_symbols, 'vfd')

Value ops:  2*ADD + 2*DIV + 15*MUL + 4*POW + 35*SUB
Jacob ops:  63*ADD + 60*DIV + 336*MUL + 9*NEG + 84*POW + 678*SUB


# Edge Edge Distance

In [15]:
e0 = a - b
e1 = c - d
ee_norm = e0.cross(e1).normalized()
ee_distance = simplify((c - a).dot(ee_norm))
ee_distance

(-(a_0 - c_0)*((a_1 - b_1)*(c_2 - d_2) - (a_2 - b_2)*(c_1 - d_1)) + (a_1 - c_1)*((a_0 - b_0)*(c_2 - d_2) - (a_2 - b_2)*(c_0 - d_0)) - (a_2 - c_2)*((a_0 - b_0)*(c_1 - d_1) - (a_1 - b_1)*(c_0 - d_0)))/sqrt(((a_0 - b_0)*(c_1 - d_1) - (a_1 - b_1)*(c_0 - d_0))**2 + ((a_0 - b_0)*(c_2 - d_2) - (a_2 - b_2)*(c_0 - d_0))**2 + ((a_1 - b_1)*(c_2 - d_2) - (a_2 - b_2)*(c_1 - d_1))**2)

In [16]:
generate_code_for(ee_distance, free_symbols, 'eed')

Value ops:  2*ADD + 2*DIV + 15*MUL + 4*POW + 35*SUB
Jacob ops:  78*ADD + 72*DIV + 420*MUL + 12*NEG + 96*POW + 834*SUB


# Vertex Edge Distance

In [17]:
ev_distance = simplify(e0.normalized().cross(c - a).norm())
ev_distance

sqrt(((a_0 - b_0)*(a_1 - c_1) - (a_0 - c_0)*(a_1 - b_1))**2 + ((a_0 - b_0)*(a_2 - c_2) - (a_0 - c_0)*(a_2 - b_2))**2 + ((a_1 - b_1)*(a_2 - c_2) - (a_1 - c_1)*(a_2 - b_2))**2)/sqrt((a_0 - b_0)**2 + (a_1 - b_1)**2 + (a_2 - b_2)**2)

In [18]:
ev_free = free_symbols[:9]
generate_code_for(ev_distance, ev_free, 'evd')

Value ops:  4*ADD + 3*DIV + 6*MUL + 8*POW + 18*SUB
Jacob ops:  75*ADD + 63*DIV + 195*MUL + 120*POW + 384*SUB


# Vertex Vertex Distance

In [19]:
vv_dist = simplify(e0.norm())
vv_dist

sqrt((a_0 - b_0)**2 + (a_1 - b_1)**2 + (a_2 - b_2)**2)

In [20]:
generate_code_for(vv_dist, free_symbols[:6], 'vvd')

Value ops:  2*ADD + DIV + 4*POW + 3*SUB
Jacob ops:  12*ADD + 12*DIV + 24*POW + 24*SUB
